In [2]:
import pandas as pd
import numpy as np

In [3]:
base = pd.read_csv('./input/basestructuredf.csv')
sales = pd.read_csv('./input/cleanedmonthly.csv')
test = pd.read_csv('./input/test.csv')
items = pd.read_csv('./input/items.csv')

In [4]:
items.drop(columns='item_name', inplace=True)

In [5]:
combined = base.merge(sales, 'outer', on=['year', 'date_block_num','month','shop_id', 'item_id'])

In [6]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18648420 entries, 0 to 18648419
Data columns (total 7 columns):
 #   Column                Dtype  
---  ------                -----  
 0   shop_id               int64  
 1   item_id               int64  
 2   date_block_num        int64  
 3   month                 int64  
 4   year                  int64  
 5   item_cnt_month        float64
 6   median_monthly_price  float64
dtypes: float64(2), int64(5)
memory usage: 1.1 GB


In [7]:
def fillnavals_price_item_cnt(df):
    df.median_monthly_price = df.groupby(['shop_id', 'item_id']).median_monthly_price.fillna(method='ffill')
    df.median_monthly_price = df.groupby(['shop_id', 'item_id']).median_monthly_price.fillna(value=0)
    df.fillna(value=0, inplace=True)

In [8]:
fillnavals_price_item_cnt(combined)

In [9]:
combined.sample(10)

shop_id  item_id  date_block_num  month  year  item_cnt_month  \
8869872        31    21614              32      9  2015             0.0   
14794746       49     6855               1      2  2013             0.0   
5005598        18     5814               3      4  2013             0.0   
5906812        21     6507               2      3  2013             0.0   
11475           2      539              30      7  2015             0.0   
8217449        28    11097               9     10  2013             0.0   
1487723         5     7608              13      2  2014             0.0   
5112764        18    11252              34     11  2015             0.0   
12057302       42     3247              12      1  2014             0.0   
16241700       53    12884              20      9  2014             0.0   

          median_monthly_price  
8869872                   99.0  
14794746                   0.0  
5005598                    0.0  
5906812                    0.0  
11475                      0.0  
8217449                    0.0  
1487723                    0.0  
5112764                  449.0  
12057302                   0.0  
16241700                   0.0

In [10]:
def months_since_last_sale(df):
    df['date_block_with_sale'] = df.apply(lambda o: o.date_block_num + 1 if o.item_cnt_month > 0 else None, axis=1)
    df['date_block_with_sale'] = df.groupby(['shop_id', 'item_id']).date_block_with_sale.fillna(method='ffill')
    df['date_block_with_sale'] = df.groupby(['shop_id', 'item_id']).date_block_with_sale.fillna(value=0)
    lagging_db_with_sale = df.groupby(['shop_id', 'item_id']).date_block_with_sale.shift(1)
    df['months_since_sale'] = df.date_block_num - lagging_db_with_sale
    df.drop(columns='date_block_with_sale', inplace=True)

In [11]:
months_since_last_sale(combined)

In [12]:
combined.loc[(combined.shop_id==59)& (combined.item_id==22167)]

shop_id  item_id  date_block_num  month  year  item_cnt_month  \
18648385       59    22167               0      1  2013             0.0   
18648386       59    22167               1      2  2013             0.0   
18648387       59    22167               2      3  2013             0.0   
18648388       59    22167               3      4  2013             0.0   
18648389       59    22167               4      5  2013             0.0   
18648390       59    22167               5      6  2013             0.0   
18648391       59    22167               6      7  2013             0.0   
18648392       59    22167               7      8  2013             0.0   
18648393       59    22167               8      9  2013             0.0   
18648394       59    22167               9     10  2013             1.0   
18648395       59    22167              10     11  2013             0.0   
18648396       59    22167              11     12  2013             2.0   
18648397       59    22167              12      1  2014             0.0   
18648398       59    22167              13      2  2014             0.0   
18648399       59    22167              14      3  2014             0.0   
18648400       59    22167              15      4  2014             0.0   
18648401       59    22167              16      5  2014             0.0   
18648402       59    22167              17      6  2014             1.0   
18648403       59    22167              18      7  2014             0.0   
18648404       59    22167              19      8  2014             0.0   
18648405       59    22167              20      9  2014             0.0   
18648406       59    22167              21     10  2014             0.0   
18648407       59    22167              22     11  2014             0.0   
18648408       59    22167              23     12  2014             0.0   
18648409       59    22167              24      1  2015             0.0   
18648410       59    22167              25      2  2015             0.0   
18648411       59    22167              26      3  2015             0.0   
18648412       59    22167              27      4  2015             0.0   
18648413       59    22167              28      5  2015             0.0   
18648414       59    22167              29      6  2015             0.0   
18648415       59    22167              30      7  2015             0.0   
18648416       59    22167              31      8  2015             0.0   
18648417       59    22167              32      9  2015             0.0   
18648418       59    22167              33     10  2015             0.0   
18648419       59    22167              34     11  2015             0.0   

          median_monthly_price  months_since_sale  
18648385                   0.0                NaN  
18648386                   0.0                1.0  
18648387                   0.0                2.0  
18648388                   0.0                3.0  
18648389                   0.0                4.0  
18648390                   0.0                5.0  
18648391                   0.0                6.0  
18648392                   0.0                7.0  
18648393                   0.0                8.0  
18648394                 299.0                9.0  
18648395                 299.0                0.0  
18648396                 299.0                1.0  
18648397                 299.0                0.0  
18648398                 299.0                1.0  
18648399                 299.0                2.0  
18648400                 299.0                3.0  
18648401                 299.0                4.0  
18648402                 299.0                5.0  
18648403                 299.0                0.0  
18648404                 299.0                1.0  
18648405                 299.0                2.0  
18648406                 299.0                3.0  
18648407                 299.0                4.0  
18648408                 299.0                5.0  
18648409 

In [13]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18648420 entries, 0 to 18648419
Data columns (total 8 columns):
 #   Column                Dtype  
---  ------                -----  
 0   shop_id               int64  
 1   item_id               int64  
 2   date_block_num        int64  
 3   month                 int64  
 4   year                  int64  
 5   item_cnt_month        float64
 6   median_monthly_price  float64
 7   months_since_sale     float64
dtypes: float64(3), int64(5)
memory usage: 1.3 GB


In [14]:
combined = pd.merge(left=combined, right=items, how='left', left_on='item_id', right_on='item_id')

In [15]:
combined

shop_id  item_id  date_block_num  month  year  item_cnt_month  \
0               2        1               0      1  2013             0.0   
1               2        1               1      2  2013             0.0   
2               2        1               2      3  2013             0.0   
3               2        1               3      4  2013             0.0   
4               2        1               4      5  2013             0.0   
...           ...      ...             ...    ...   ...             ...   
18648415       59    22167              30      7  2015             0.0   
18648416       59    22167              31      8  2015             0.0   
18648417       59    22167              32      9  2015             0.0   
18648418       59    22167              33     10  2015             0.0   
18648419       59    22167              34     11  2015             0.0   

          median_monthly_price  months_since_sale  item_category_id  
0                          0.0                NaN                76  
1                          0.0                1.0                76  
2                          0.0                2.0                76  
3                          0.0                3.0                76  
4                          0.0                4.0                76  
...                        ...                ...               ...  
18648415                 299.0               12.0                49  
18648416                 299.0               13.0                49  
18648417                 299.0               14.0                49  
18648418                 299.0               15.0                49  
18648419                 299.0               16.0                49  

[18648420 rows x 9 columns]

In [16]:
def add_lagging_values(df, num_months=3, attributes=['item_cnt_month', 'median_monthly_price']):
    for att in attributes:
        for i in range(num_months):
            df[f'{att} - {i+1}'] =  df.groupby(['shop_id', 'item_id'])[att].shift(i+1)
    df.drop(np.where(df.date_block_num.isin(range(num_months)))[0], inplace=True)


In [17]:
add_lagging_values(combined)

In [18]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17049984 entries, 3 to 18648419
Data columns (total 15 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   shop_id                   int64  
 1   item_id                   int64  
 2   date_block_num            int64  
 3   month                     int64  
 4   year                      int64  
 5   item_cnt_month            float64
 6   median_monthly_price      float64
 7   months_since_sale         float64
 8   item_category_id          int64  
 9   item_cnt_month - 1        float64
 10  item_cnt_month - 2        float64
 11  item_cnt_month - 3        float64
 12  median_monthly_price - 1  float64
 13  median_monthly_price - 2  float64
 14  median_monthly_price - 3  float64
dtypes: float64(9), int64(6)
memory usage: 2.0 GB


In [19]:
combined.to_csv('./input/consolidated_data.csv', index=False)